# Semelhança de documentos

A função desse notebook é criar representações para o texto dos documentos,
exibir textos semelhantes e com isso avaliar as representações.
Os textos semelhantes são um teste para a representação do texto:
se os textos mostrados não são semelhantes, então a representação não é boa;
se são semelhantes, então a representação um pouco melhor.

## Dados

Embora a proposta deste notebook um template genérico para comparação de textos,
neste notebook utilizamos o dataset das leis municipais, disponível
[aqui](https://www.kaggle.com/anapaulagomes/leis-do-municpio-de-feira-de-santana/).


In [2]:
import pandas as pd

In [3]:
laws_file = 'leis.json'

# Descomente pra usar no Google Colab
# from google.colab import files
# import os.path


# if (not os.path.isfile(laws_file)):
#     uploaded = files.upload()

In [4]:
laws = pd.read_json(laws_file)
laws.drop(['documento'], inplace=True, axis=1)
laws.describe().T

,count,unique,top,freq
titulo,6033,6033,RESOLUÇÃO Nº 487/2013,1
categoria,6033,8,Leis Ordinárias,3367
resumo,6033,4961,DISPÕE SOBRE A CONCESSÃO DO TÍTULO DE CIDADÃO ...,153
texto,6033,6029,Conteúdo obsoleto:\nEste Ato não tem mais efe...,3


In [5]:
laws.sample(10)

,titulo,categoria,resumo,texto
1055,"LEI Nº 1422, de 11 de junho de 1991",Leis Ordinárias,CONSIDERA DE UTILIDADE PÚBLICA A ASSOCIAÇÃO CO...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, ESTA..."
3676,LEI COMPLEMENTAR Nº 24/2005,Leis Complementares,MODIFICA INCISO DE ARTIGO DA LEI COMPLEMENTAR ...,"O Prefeito Municipal de Feira de Santana, Esta..."
2883,LEI Nº 2598/2005,Leis Ordinárias,DISPÕE SOBRE A REGULAMENTAÇÃO DE ESPAÇO DENTRO...,Autor: Luis Augusto de Jesus\n\nO PREFEITO MUN...
3887,"DECRETO LEGISLATIVO Nº 3, DE 21/03/1990",Decretos Legislativos,APROVA AS CONTAS DA MESA DIRETIVA DA CÂMARA MU...,Visualizar Ato:Decreto Legislativo nº 3/1990 -...
4908,"DECRETO LEGISLATIVO Nº 10, DE 20/06/1995",Decretos Legislativos,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Visualizar Ato:Decreto Legislativo nº 10/1995 ...
1925,LEI Nº 2609/2005,Leis Ordinárias,DISPÕE SOBRE A CONCESSÃO E O CANCELAMENTO DE I...,Autor: Poder Executivo\n\nO PREFEITO MUNICIPAL...
2917,"LEI Nº 3788, DE 19 DE DEZEMBRO DE 2017",Leis Ordinárias,"""Dispõe sobre o repasse de Recursos Públicos M...","O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, ESTA..."
3050,"LEI Nº 446, de 05 de agosto de 1965",Leis Ordinárias,ABRE O CRÉDITO ESPECIAL DE CR$ 20.000.000 (VIN...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, ESTA..."
2738,LEI Nº 2336/2002,Leis Ordinárias,OBRIGA O USO DE BALANÇA PARA REVENDA DO GÁS LI...,Autor: Moacir Lima dos Santos\n\nO PREFEITO MU...
1883,LEI Nº 2003/98,Leis Ordinárias,"INSTITUI A CARTEIRA DE SAÚDE ESCOLAR, E DÁ OUT...","O PREFEITO MUNCIPAL DE FEIRA DE SANTANA, Estad..."


In [6]:
# Exemplo de texto de lei
print(laws.loc[len(laws)-1, 'texto'])

A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:

Art. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:

"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.

§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."

"Art. 33 Compete, privativamente, ao Vice-Presidente:"

"Art. 36 ...

I - ...

e) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presidente.

II - ...



## Comparando documentos: representação e calculo de similaridade

Para comparar quão parecido são dois documentos,
primeiro temos que transformar estes documentos 
para uma representação que o computador consiga calcular alguma coisa a respeito.
Existem alguns métodos para isto.
Neste notebook temos 3: TF, TF-IDF e vetores de palavras.
Para calcular a similaridade, também existem alguns métodos diferentes.
Utilizamos similaridade do cosseno.

### Term Frequency (TF)

A primeira representação construída é bastante ingênua:
apenas conta a quantidade de vezes que cada palavra apareceu em cada texto
e atribui um vetor pra esse texto.
Cada posição do vetor é uma palavra
e cada valor representa quantas vezes essa palavra apareceu no dado texto.
Todos os textos, portanto, são representados por uma matriz
de dimensões _m_ x _n_, onde _m_ é o número de textos 
e _n_ é o número de palavras únicas (tamanho do vocabulário).

In [7]:
from scripts.parsers import clean_text
laws['texto_limpo'] = laws['texto'].apply(clean_text)

[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Gera matriz de documentos
vectorizer = CountVectorizer()
tf_representation = vectorizer.fit_transform(laws['texto_limpo'])
tf_representation

Com a matriz de documentos ~literalmente~ em mãos,
vamos calcular a similaridade entre dois textos.
A similaridade é calculada pela similaridade do cosseno (ver algebra linear).
Existem outras medidas pra calcular similaridade / distância.
Uma discussão sobre isso [aqui](https://cmry.github.io/notes/euclidean-v-cosine).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

cos_sim_matrix = cosine_similarity(tf_representation, dense_output=True)
# sorts ascending, per row, 
# the indexes of the documents according to their cossine similarity
cos_sim_argsort = np.argsort(cos_sim_matrix) 

In [ ]:
most_similar_indexes_tf = cos_sim_argsort[:,-2] # -1 is the same text
tf_similarities = [cos_sim_matrix[i, ind] for i, ind in enumerate(most_similar_indexes_tf)] 

In [ ]:
def print_laws(original_law_index, compared_law_index: int):
    print(f'- - - LEI: {original_law_index}: - - -\n\n')
    print(laws.loc[original_law_index, 'texto'])
    print(f'\n\n- - - LEI COMPARADA: {compared_law_index} - - -\n\n')
    print(laws.loc[compared_law_index, 'texto'])

In [ ]:
max_sim_overall = np.max(tf_similarities)
print(f'Maior similaridade entre duas Leis: {max_sim_overall}')

original_law_index = np.argmax(tf_similarities)
most_similar_law_index = most_similar_indexes_tf[original_law_index]

print_laws(original_law_index, most_similar_law_index)

As leis 5949 e 6026 são idênticas.
Opa! Lei 13 e Lei 118 são a mesma lei, com 3 dias de diferença. Por que existe isso?

### TF-IDF

Outra representação possível para os textos é TF-IDF. 
Ainda contamos a frequência de cada termo (TF), 
porém ponderamos esta frequência pela raridade da palavra, 
medida pela frequência inversa que ela aparece nos documentos
(Inverse Document Frequency). 
Ou seja, quanto mais rara é a palavra no corpus, 
mais ela caracteriza o texto em que ela aparece, maior será o peso dela.

In [ ]:
transformer = TfidfTransformer()
tfidf_representation = transformer.fit_transform(tf_representation)

cos_sim_tfidf = cosine_similarity(tfidf_representation, dense_output=True)
cos_sim_tfidf_sorted_idxs = np.argsort(cos_sim_tfidf)

tfidf_representation

In [ ]:
most_similar_law_idx = cos_sim_tfidf_sorted_idxs[original_law_index, -2]
tfidf_similarity = cos_sim_tfidf[original_law_index, most_similar_law_idx]

print("Dada a mesma lei anterior, "
    f"similaridade com TF-IDF é: {tfidf_similarity}")
print_laws(original_law_index, most_similar_law_idx)

Mostram a mesma lei. O que faz sentido, já que as leis são idênticas.

In [ ]:
most_similar_indexes_tfidf = cos_sim_tfidf_sorted_idxs[:,-2]

same_result = (most_similar_indexes_tf == most_similar_indexes_tfidf)
print(same_result)
print(f"Concordam em {sum(same_result) / len(same_result) * 100}% dos resultados")

A lei mais semelhante de acordo com TF e TF-IDF é a mesma 40% das vezes.

Vamos dar uma olhada em algumas leis onde os resultados diferem,
para ter uma intuição sobre qual representação é melhor para as leis.

Vamo sortear indices aleatorios desse vetor e 
ler as leis que eles representam e as similaridades

In [ ]:
different_result_indexes = [i for i, _ in enumerate(same_result) if not same_result[i]]
comparisons_count = 10
drafted_indexes = np.random.randint(0, high=len(different_result_indexes)-1, size=comparisons_count)
drafted_laws = [different_result_indexes[i] for i in drafted_indexes]
print(drafted_laws)

In [ ]:
for i in drafted_laws:
    print('\n\nCOMPARACAO UTILIZANDO TF:\n\n')
    print_laws(i, most_similar_indexes_tf[i])
    print('\n\nCOMPARACAO UTILIZANDO TF-IDF:\n\n')
    print_laws(i, most_similar_indexes_tfidf[i])

Como é um sorteio, 
cada vez que você rodar esse notebook vai ter resultados diferentes.
Fique a vontade pra fazer um PR com a comparação de leis diferentes.
Abaixo estão comparações da primeira vez que rodei

### Lei 1018
Lei 1018 é sobre proibição de homenagens a condenados por corrupção.

TF trouxe uma lei sobre tornar uma associação pública.

TF-IDF trouxe uma lei sobre evento de comemoração de adoção animal.

Todas duas erraram.

### Lei 5776
Lei 5776 sobre pagamento servidor público.

TF trouxe: leitura da bíblia na abertura da câmara 
(que bizarro, diga-se de passagem).

TF-IDF: aposentadoria diretor valor vencimento etc.

Ambas as leis parecem ter sido trazidas como semelhantes
por causa dos nomes próprios contidos nas leis.

### Lei 2789
Lei 2789 (mil anos da revolução francesa) sobre obrigatoriedade 
de um servidor formado em primeiros socorros em escolas. 

TF: faço saber inkaba instituto de karate. 

TF-IDF: faço saber associação estrela jaco. 

Novamente as semelhanças são os nomes próprios nas leis.

### Lei 1772
Lei 1772: faço saber sindicato trabalhadores rurais.

TF: faço saber associação profissionais sexo.

TF-IDF: faço saber associação pequenos agricultures apaeb -
a rua da sede é a mesma da lei comparada. 

TF-IDF se saiu melhor nessa.
Os nomes das pessoas em TF eram os mesmos da Lei,
mas em TF-IDF não.
O fator decisivo aqui foi o nome da rua, que era o mesmo.
Ponto pra TF-IDF.

### Lei 530
Lei 530: faço saber igreja ministerio pentecostal fogo gloria.
rua volta redonda bairro campo limpo.

TF: faço saber instituto nobre sede rodovia br km cis.
nomes das pessoas iguais.

TF-IDF: faço saber igreja evangelica pentecostal monte carmelo
rua espassonavel bairro george americo.
prefeitos diferentes.

### Lei 4810
Lei: comenda.
nomes: godofredo rebell figueiredo filho,
raymundo luiz oliveira lopes.

TF: comenda.
nomes: godofredo rebello figueiredo filho,
nilton bellas vieira.

TF-IDF: comenda.
nomes: godofredo rebello figueiredo filho,
raimundo antonio carneiro pinto.

As duas acertaram

### Lei 5238 
Lei 5238: promulgação de novas vias públicas.
A via por TF-IDF passa por mais ruas iguais.

### Lei 5383
Lei promulga academia de ginástica.
TF: promulga empresas serviço funerario.
TF-IDF: promulga novos aparelhos de ginástica.

### Outras
As outras leis eram: "_visualizar legislativo ba_".
Ambas trouxeram textos idênticos.

Ok! Massa! Funciona!

Pelos resultados acima, TF-IDF se saiu melhor. 
Inclusive pra retornar semelhança 
por nomes de ruas e de bairros,
que é o que a gente quer pras buscas.

Mesmo com 28k features,
o resultado foi bastante rápido. 
Caso tivessemos um corpus maior,
poderíamos ainda usar PCA pra reduzir as dimensões
e ainda assim calcular a similaridade
mantendo as relações entre os documentos.

### Vetor de palavras - word embedding

A ideia dessa representação é 
criar um vetor pra cada palavra,
com base nas palavras vizinhas.

Isto vem da hipótese linguística distribucional:
uma palavra é parecida com outra se
suas vizinhas são as mesmas.

Ou ainda:
"conhecerás a palavra pelas compainhas que ela mantém".

Na prática, vamos utilizar um método
baseado nesta hipótese chamado
[word2vec](https://arxiv.org/pdf/1301.3781.pdf).

Cada palavra é representada por um vetor
de _n_ dimensões. Estes vetores são
inicializados de forma aleatória.
A seguir, o modelo tenta prever uma palavra
de acordo com as anteriores (CBOW) ou as 
palavras vizinhas (skip-gram) e vai ajustando
os valores de cada vetor de acordo.

Ao final do processo, os vetores possuem
alguma informação semântica sobre as palavras.
Por exemplo, o vetor de "rei" e "rainha" estarão
próximos e distantes de "ônibus" e "avião",
palavras utilizadas em outro contexto (vizinhança).

Isto possibilita operações aritiméticas com as
palavras. Por exemplo:

`rei` - `homem` + `mulher` = `rainha`

Mais informações sobre word2vec nas duas primeiras
[aulas](https://www.youtube.com/watch?v=8rXD5-xhemo&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z)
deste curso. Caso você tenha um material que explique
word2vec, especialmente em português, manda pra gente
ou faz um PR! Adoraríamos essa contribuição!


In [20]:
from gensim.models import Word2Vec


corpus = laws["texto_limpo"].apply(lambda x: x.split())
model_w2v = Word2Vec(sentences=corpus, window=5, min_count=5, workers=8)
model_w2v.save("models/word2vec.model")

### Palavras semelhantes

Uma das vantagens desse modelo é que
podemos explorar a semelhança entre palavras.

No texto das Leis, quais as palavras mais semelhantes
a `educação` ou `saúde`, por exemplo?

Você pode explorar outras semelhanças modificando
o código da próxima célula:

In [43]:
model.most_similar("transporte", topn=10)

[('coletivo', 0.887986421585083),
 ('passageiros', 0.8504563570022583),
 ('alternativo', 0.8208197951316833),
 ('cargas', 0.7519239187240601),
 ('táxi', 0.7251712083816528),
 ('convencional', 0.7149850130081177),
 ('veículos', 0.7121580839157104),
 ('individual', 0.7088261842727661),
 ('coletivos', 0.7059678435325623),
 ('stiac', 0.7000055313110352)]

Semelhantes a `educação`:
saúde, escolar, seduc, especialistas
executiva, escolarização, cultura
prevenção, professores, endemias.

Semelhantes a `saúde`:
educação, competirá, vigilância,
seduc, médico-hospitalar, básico
segunraça, sus, incumbida, rede

Semelhantes a `segurança`:
higiene, alimentar, nutricional,
conforto, coletiva, zelar,
garantir, engenharia, visando, melhor.

Semelhantes a `transporte`:
coletivo, passageiros, alternativo,
cargas, táxi, convencional, veículos,
individual, coeltivos, stiac.

## Construindo a representação das Leis

O Gensim também traz uma ferramenta para
gerar embeddings de textos inteiros, chamada
[Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec).
Esta técnica utiliza uma ideia semelhante ao Word2Vec,
com adaptações para cobrir um texto inteiro.

In [44]:
from gensim.models.doc2vec import Doc2Vec


model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
# PS: Tendo um compilador C, você consegue tornar o Doc2Vec em até 70x mais rápido

NameError: name 'documents' is not defined

In [ ]:
most_similar_indexes_word_embedding = [idx for idx in np.argsort(word_embedding_cosine_similarity)[:,-2]]

In [ ]:
# Diferenca de vetores pra TFIDF eh bem maior.
# Vamos samplear algumas dessas diferencas e 
# mostrar uma lei q tanto TFIDF como Count erraram
different_result_from_tfidf = (most_similar_indexes_word_embedding != most_similar_indexes_tfidf)
print("Porcentagem de resultados diferentes de TFIDF: "
    f"{sum(different_result_from_tfidf)*100 / len(different_result_from_tfidf)}")

In [ ]:
drafted_laws_index = np.random.randint(len(different_result_from_tfidf), size=10)
for i in drafted_laws_index:
    if different_result_from_tfidf[i]:
        print('\n\nVETOR DE PALAVRAS:\n\n')
        print_laws(i, most_similar_indexes_word_embedding[i])
        print('\n\nTF-IDF:\n\n')
        print_laws(i, most_similar_indexes_tfidf[i])

Parece que TF-IDF é um pouco melhor na comparação de leis,
pq traz resultados mais relevantes quando comparados nome de bairros e ruas.
No entanto, a forma vetorizada parece ser boa pra reconhecer formatos da Lei em geral,
uma especie de POS, reconhecendo que existe uma entidade alí ou verbo etc.
Ao menos foi minha impressão.
Cabe mais investigação a respeito.

Vale salientar que a qualidade dos vetores parece não estar tão boa.
Vide a semelhança de palavras. Como fazer pra consertar isso?
Seria muito interessante corrigir isso pra ver as palavras mais semelhantes à
educação,saúde etc e também pra visualizar com tsne os clusters gerados a partir daí.

## Outras opções
### Indexar

Há outras formas de indexar os documentos e de recuperar, também simples.
Uma outra forma de indexar, por exemplo,
é fazer um vetor pra cada palavra contando as palavras vizinhas.
E depois, o vetor do documento seria a soma dos vetores das palavras.
É uma forma interessante porque pode gerar visualizações interessantes
entre a similaridade das palavras.
Por exemplo, no corpus das Leis Municipais,
a quais palavras EDUCAÇÃO mais se assemelha? Ou SAÚDE? Etc.

Outra forma é contar n-gramas - por exemplo, bi-gramas:
duas palavras juntas formando um token.
Dessa forma, você possui uma matriz maior
e de certa forma uma relação entre a sequencialidade das palavras,
que pode ser útil pra nomes de pessoas e bairros, como citado acima.

### Recuperar

Outra forma de recuperar é por local sensitive hashing.
Divide em vários planos múltiplas vezes
e retorna os resultados que estão na mesma região da query.
No entanto, o corpus não é grande o suficiente pra precisar essa estratégia,
que é mais pra grandes corpora.
O método acima (calcular a simlaridade cosseno e retornar os maiores valores)
é rápido o suficiente pra parecer instantâneo.
Talvez com uma demanda mais alta pelo servidor venha a necessidade de aumentar
a velocidade da busca, porém por enquanto não é o caso.
Mais sobre recuperação: Google lançou [novo método]
(https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)
e uma lib pra isso agora, dia 28 de Julho.

### Avaliação

Com múltiplas formas de indexar e recuperar vem o dilema:
como avaliar se uma é melhor que a outra?
Repetir o processo acima pra todas as opções?
Isto é, mostrar N melhores resultados e comparar manualmente?
Ou colocar labels em algumas leis? Ex: essa lei trata disso, com tais entidades.
Checar formas de avaliação.
Se tivesse em produção, podia avaliar por CTR por ex, mas não é o caso